In [1]:
import bs4 as bs
import urllib.request

In [2]:
source_lst = ['http://www.wuhanrc.org.cn/info/1003/2687.htm',
             'http://www.wuhanrc.org.cn/info/1003/2696.htm',
             'http://www.wuhanrc.org.cn/info/1003/2700.htm',
             'http://www.wuhanrc.org.cn/info/1003/2703.htm',
             'http://www.wuhanrc.org.cn/info/1003/2704.htm',
             'http://www.wuhanrc.org.cn/info/1003/2729.htm',
             'http://www.wuhanrc.org.cn/info/1003/2737.htm',
             'http://www.wuhanrc.org.cn/info/1003/2748.htm',
             'http://www.wuhanrc.org.cn/info/1003/2752.htm',
             'http://www.wuhanrc.org.cn/info/1003/2760.htm',
             'http://www.wuhanrc.org.cn/info/1003/2764.htm']

In [3]:
s_lst = []
for s in source_lst:
    source = urllib.request.urlopen(s).read()
    soup = bs.BeautifulSoup(source,'html.parser')
    tbl_row = soup.table.find_all('tr')
    for tr in tbl_row[1:]:
        td = tr.find_all('td')
        row = [i.text for i in td]
        s_lst.append(row)


IncompleteRead: IncompleteRead(179762 bytes read, 293732 more expected)

In [ ]:
for i,row in enumerate(s_lst):
    if len(row) == 4:
        s_lst[i]=[s_lst[i-1][0]] + row

In [ ]:
import pandas as pd

In [ ]:
donation = pd.DataFrame(data = s_lst, columns = ['接收单位', '品名', '规格', '计量单位', '数量'])

In [ ]:
protect_suit = donation[(donation['品名'].str.contains('防护服')) & (donation['品名'] != '非医用防护服')\
         &(donation['品名'] != '居家隔离防护服包')].copy()

In [ ]:
protect_suit['Tao'] = protect_suit['规格'].str.extract('(\d+套)', expand=True)[0].str.extract('(\d+)', expand=True)
protect_suit['Jian'] = protect_suit['规格'].str.extract('(\d+件)', expand=True)[0].str.extract('(\d+)', expand=True)


In [ ]:
import numpy as np
protect_suit['特殊单位'] = np.where(protect_suit['Tao'].notnull() == True, protect_suit['Tao'], protect_suit['Jian'])


In [ ]:
protect_suit['特殊单位'] = np.where(protect_suit['规格'] == '600PLNS \xa0 2XL号', 100, protect_suit['特殊单位'])
protect_suit['特殊单位'] = np.where(protect_suit['规格'] == '600PLNS \xa0 LG号', 100, protect_suit['特殊单位'])

In [ ]:
protect_suit['特殊单位'] = protect_suit['特殊单位'].fillna(0).astype('int64')
protect_suit['数量'] = protect_suit['数量'].astype('int64')

In [ ]:
protect_suit['接收单位类别'] = np.where(protect_suit['接收单位'].str.contains('院'), '医院', '行政')
protect_suit['接收单位类别'] = np.where(protect_suit['接收单位'].str.contains('方舱'), '医院', protect_suit['接收单位类别'])

In [ ]:
protect_suit['处理后数量'] = np.where(protect_suit['特殊单位'] == 0, protect_suit['数量'], \
                                 protect_suit['特殊单位']*protect_suit['数量'])

protect_suit['处理后数量'] = np.where((protect_suit['接收单位类别'] == '行政')&\
             (protect_suit['品名'] == '防护服Plus600型')&(protect_suit['计量单位'] == '套'), \
                             protect_suit['处理后数量']/protect_suit['特殊单位'],\
                                 protect_suit['处理后数量'])

protect_suit['处理后数量'] = np.where((protect_suit['品名']=='防护服')&(protect_suit['处理后数量']>10000),\
                                 protect_suit['处理后数量']/protect_suit['特殊单位'],\
                                 protect_suit['处理后数量'])
protect_suit['处理后数量'] = np.where((protect_suit['品名']=='医用防护服')&(protect_suit['处理后数量']>10000),\
                                 protect_suit['处理后数量']/protect_suit['特殊单位'],\
                                 protect_suit['处理后数量'])

In [ ]:
protect_suit.head()

In [ ]:
plot1 = protect_suit.groupby('接收单位类别').sum()['处理后数量'].reset_index()

In [ ]:
plot2_pre = protect_suit.groupby(['接收单位类别','品名']).sum().reset_index()
plot2 = plot2_pre[plot2_pre['接收单位类别'] == '行政']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
sns.set(font_scale=2)
sns.set_palette(sns.color_palette("Set2"))
plt.rcParams['font.sans-serif'] = ['SimHei']

In [ ]:
g = sns.barplot(x="接收单位类别", y="处理后数量",data=plot1)
g.set_title("防护服领取情况医院（名称中包含‘院’或方舱）\n VS行政机构（因为没有企业）")
plt.show()

In [ ]:
sns.set_palette(sns.color_palette("Set2"))
g2 = sns.barplot(y="品名", x="处理后数量",data=plot2.sort_values(by = '处理后数量',ascending=False))
g2.set_title("行政领取具体by 品名")
plt.show()

In [ ]:
protect_suit[(protect_suit['接收单位类别']=='行政') & (protect_suit['品名']=='防护服')\
             &(protect_suit['处理后数量']>2000)]

In [ ]:
plot3 = protect_suit.groupby('接收单位').sum().sort_values(by = '处理后数量',ascending=False).reset_index().head(10)

In [ ]:
g3 = sns.barplot(y="接收单位", x="处理后数量",data=plot3)
g3.set_title("领取防护服最多的单位")
plt.show()

In [ ]:
plt.clf()